Компании оказывающие телекоммуникационные услуги, интернет-провайдеры, компании платного телевидения, страховые компании и службы мониторинга сигнализации часто используют клиентский анализ и показатели оттока клиентов в качестве одной из своих ключевых бизнес-метрик, поскольку стоимость удержания существующего клиента намного меньше, чем приобретение нового. Компании из этих секторов часто имеют филиалы по обслуживанию клиентов, которые пытаются удержать клиентов, потому что лояльные долгосрочные клиенты могут принести компании гораздо больше дохода, чем вновь набранные клиенты.

Компании обычно проводят различие между добровольным оттоком и ненамеренным оттоком. Добровольный отток происходит из-за решения клиента перейти на другую компанию или поставщика услуг, ненамеренный отток происходит из-за обстоятельств, таких как переезд или смерть. В большинстве случаев ненамеренные причины отказа от услуг компании исключаются из аналитических моделей. Аналитики, как правило, концентрируются на добровольном оттоке, поскольку это обычно происходит из-за факторов взаимоотношений компании и клиента, которые компании могут контролировать, таких, например, как стоимость услуг или оказание поддержки пользователей.

Предиктивная аналитика использует специальные модели, которые прогнозируют отток клиентов, оценивая риск отказа абонента от услуг компании. Поскольку эти модели генерируют небольшой приоритетный список потенциальных перебежчиков, они эффективны при разработке маркетинговых программ удержания клиентов на подмножестве клиентской базы, которые наиболее склонны к отказу от услуг.

Задача - как можно точнее предсказать, откажется ли абонент от услуг компании на следующий месяц. Вывести вероятность отказа, то есть вещественное число на промежутке от 0 до 1 для каждого абонента из тестового набора.

#### Дано: тренировочные данные (train.csv), тестовые данные (test.csv).
#### Цель: для каждого абонента предсказать колонку `Leave_Next_Month` которая есть в тренировочном файле, но отсутствует в тестовом файле. Значение должно находится на промежутке от 0 до 1, где 0 - абонент точно не уйдет в следующем месяце, а 1 - абонент точно уйдет в следующем месяце.

# Предсказания с помощью модели градиентного бустинга

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# шаг 1 - загрузка датасетов и выделение целевой и зависимых переменных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train

,id,Leave_Next_Month,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
0,104,1,F,0,N,N,2,Y,N,Fiber optic,...,N,N,N,N,N,Per_month,Y,Check_electron,70.70,151.65
1,107,0,F,0,N,N,10,N,No_phone,DSL,...,N,N,N,N,N,Per_month,N,Check_mailed,29.75,301.9
2,108,1,F,0,Y,N,28,Y,Y,Fiber optic,...,N,Y,Y,Y,Y,Per_month,Y,Check_electron,104.80,3046.05
3,109,0,M,0,N,Y,62,Y,N,DSL,...,Y,N,N,N,N,Year-1,N,Bank_transfer_auto,56.15,3487.95
4,111,0,M,0,N,N,16,Y,N,N,...,No_internet,No_internet,No_internet,No_internet,No_internet,Year-2,N,Credit_card_auto,18.95,326.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516,7135,0,M,0,N,N,19,Y,N,Fiber optic,...,N,N,N,Y,N,Per_month,Y,Bank_transfer_auto,78.70,1495.1
3517,7136,0,F,0,N,N,12,N,No_phone,DSL,...,Y,Y,Y,Y,Y,Year-1,N,Check_electron,60.65,743.3
3518,7138,0,M,0,Y,Y,24,Y,Y,DSL,...,N,Y,Y,Y,Y,Year-1,Y,Check_mailed,84.80,1990.5
3519,7140,0,F,0,Y,Y,11,N,No_phone,DSL,...,N,N,N,N,N,Per_month,Y,Check_electron,29.60,346.45


In [ ]:
# шаг 2 - предобработка данных
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in train.columns:
  if train[col].dtype != 'int64':
      train[col] = le.fit_transform(train[col])
        
for col in test.columns:
  if test[col].dtype != 'int64':
      test[col] = le.fit_transform(test[col])

train

,id,Leave_Next_Month,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
0,104,1,0,0,0,0,2,1,0,1,...,0,0,0,0,0,0,1,1,574,480
1,107,0,0,0,0,0,10,0,1,0,...,0,0,0,0,0,0,0,2,122,1392
2,108,1,0,0,1,0,28,1,2,1,...,0,2,2,2,2,0,1,1,1125,1412
3,109,0,1,0,0,1,62,1,0,0,...,2,0,0,0,0,1,0,0,382,1596
4,111,0,1,0,0,0,16,1,0,2,...,1,1,1,1,1,2,0,3,5,1504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516,7135,0,1,0,0,0,19,1,0,1,...,0,0,0,2,0,0,1,0,692,462
3517,7136,0,0,0,0,0,12,0,1,0,...,2,2,2,2,2,1,0,1,440,2977
3518,7138,0,1,0,1,1,24,1,2,0,...,0,2,2,2,2,1,1,2,795,855
3519,7140,0,0,0,1,1,11,0,1,0,...,0,0,0,0,0,0,1,1,119,1579


In [ ]:
y_tr = train['Leave_Next_Month']
x_tr = train.drop(['Leave_Next_Month'], axis=1)

In [ ]:
# шаг 3 - деление train выборки на тестовую и обучающую
x_train, x_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.3, random_state=0)
x_train

,id,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,Online_Security,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
2433,4921,1,0,1,0,56,0,1,0,0,0,2,2,2,2,1,1,0,343,1365
850,1755,0,0,1,1,3,1,0,0,0,0,0,0,0,0,0,0,2,218,314
135,355,0,0,1,1,67,0,1,0,0,2,2,2,2,2,2,1,1,419,1821
849,1751,1,1,0,0,1,1,0,1,0,0,0,0,2,0,0,1,1,700,3090
820,1683,0,0,1,1,6,1,2,0,0,0,0,0,0,0,0,1,3,269,1265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1721,0,0,0,0,35,1,2,1,0,0,0,0,2,0,0,1,1,827,1407
3264,6584,1,0,0,0,37,1,2,0,2,2,2,2,2,2,1,1,0,898,1539
1653,3294,0,1,1,0,43,0,1,0,0,2,2,0,2,2,0,0,3,371,1053
2607,5266,1,0,0,0,61,1,2,1,2,0,2,0,2,2,1,0,1,1110,2657


In [ ]:
y_test

378     0
1225    0
575     0
3395    1
467     1
       ..
1388    0
1567    1
1418    0
269     0
3423    0
Name: Leave_Next_Month, Length: 1057, dtype: int64

In [ ]:
# шаг 4 - обучение модели
xgb_model = xgb.XGBClassifier(booster='dart',
                                  max_depth=3,
                                  gamma=0,eval_metric=['auc', 'error'])
xgb_model.fit(x_train, y_train)

XGBClassifier(booster='dart', eval_metric=['auc', 'error'])

In [ ]:
# шаг 5 - предсказание только на данных файла train
y_pred = xgb_model.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
# шаг 6 - оценка модели, показатели достаточно неплохие, можно использовать на новых данных файла test
accuracy = accuracy_score(y_test, np.round(y_pred))
auc = roc_auc_score(y_test, y_pred)
print('Accuracy: {:.2f} %, ROC AUC: {:.2f}'.format(100*accuracy, auc))

Accuracy: 80.32 %, ROC AUC: 0.71


In [ ]:
# шаг 7 - применение модели к тестовой выборке
pred_res = xgb_model.predict(test)
pred_res_proba = xgb_model.predict_proba(test)[:,1]
pred_res_proba

array([0.7258171 , 0.01390163, 0.21514575, ..., 0.00430368, 0.01334254,
       0.1284122 ], dtype=float32)

In [ ]:
test.insert(1, column='Leave_Next_Month', value=np.round(pred_res_proba,2))
test

,id,Leave_Next_Month,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
0,100,0.73,0,0,1,0,1,0,1,0,...,2,0,0,0,0,0,1,1,124,1252
1,101,0.01,1,0,0,0,34,1,0,0,...,0,2,0,0,0,1,0,2,405,725
2,102,0.22,1,0,0,0,2,1,0,0,...,2,0,0,0,0,0,1,2,354,83
3,103,0.01,1,0,0,0,45,0,1,0,...,0,2,2,0,0,1,0,0,212,697
4,105,0.41,0,0,0,0,8,1,2,1,...,0,2,0,2,2,0,1,1,1063,3132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,7132,0.93,1,1,0,0,1,1,2,1,...,0,0,0,0,0,0,1,1,671,2968
3518,7134,0.18,0,0,0,0,67,1,2,1,...,2,2,0,2,0,0,1,3,1114,2779
3519,7137,0.00,0,0,0,0,72,1,0,2,...,1,1,1,1,1,2,1,0,50,390
3520,7139,0.01,0,0,1,1,72,1,2,1,...,2,2,0,2,2,1,1,3,1118,2919


#### Сохраним результат в файл

In [ ]:
result = test.copy()
result = result.drop(result.columns[2:], axis=1)
result.to_csv('submission.csv', index=False)
result

,id,Leave_Next_Month
0,100,0.73
1,101,0.01
2,102,0.22
3,103,0.01
4,105,0.41
...,...,...
3517,7132,0.93
3518,7134,0.18
3519,7137,0.00
3520,7139,0.01
